# collaborative_filtering

- **Goal: Personalized book recommendations**
- Needs:
    1. Users with smililar taste in books
    2. A way to predict what we will like based on what they like

## Datasets

- **Goodreads book data**
    - Scraped by researchers at UCSD
    - Available at this [link](https://sites.google.com/eng.ucsd.edu/ucsdbookgraph)

## Project steps

1. Find similar users
2. Create Matrix
3. Recommend books

In [1]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Download these files

# books_titles.json
# https://drive.google.com/file/d/1Iqv9TROqNgYbUDijSaDegv4EPpxO97t3/view?usp=sharing

# goodreads_interactions.csv
# https://drive.google.com/open?id=1zmylV7XW2dfQVCLeg1LbllfQtHD2KUon

# book_id_map.csv
# https://drive.google.com/uc?id=1CHTAaNwyzvbi1TR08MJrJ03BxA266Yxr

# liked_books.csv
# https://drive.google.com/file/d/1dhPhfD5hAOJjrdf8JhvbOPxDpF4qWYnb/view?usp=sharing

In [3]:
# copy files to runtime
!cp /content/drive/MyDrive/Colab_Notebooks/Projects/books_recommendation/datasets/books_titles.json .
!cp /content/drive/MyDrive/Colab_Notebooks/Projects/books_recommendation/datasets/goodreads_interactions.csv .
!cp /content/drive/MyDrive/Colab_Notebooks/Projects/books_recommendation/datasets/book_id_map.csv .
!cp /content/drive/MyDrive/Colab_Notebooks/Projects/books_recommendation/datasets/liked_books.csv .

### Reading in books we like

In [4]:
# Load 'liked_books.csv' file

import pandas as pd

my_books = pd.read_csv("liked_books.csv", index_col=0)

In [5]:
my_books

,user_id,book_id,rating,title
0,-1,2517439,5,"The Forever War (The Forever War, #1)"
1,-1,113576,5,The Smartest Guys in the Room: The Amazing Ris...
2,-1,35100,5,Battle Cry of Freedom
3,-1,228221,5,The Mask of Command
5,-1,17662739,5,"2001: A Space Odyssey (Space Odyssey, #1)"
6,-1,356824,5,India After Gandhi: The History of the World's...
7,-1,12125412,5,The Lady or the Tiger?: and Other Logic Puzzles
8,-1,139069,5,Endurance: Shackleton's Incredible Voyage
10,-1,76680,5,"Foundation (Foundation, #1)"
11,-1,1898,5,Into Thin Air: A Personal Account of the Mount...


In [6]:
my_books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 532
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  27 non-null     int64 
 1   book_id  27 non-null     int64 
 2   rating   27 non-null     int64 
 3   title    27 non-null     object
dtypes: int64(3), object(1)
memory usage: 1.1+ KB


In [7]:
# convert 'book_id' column to str type
my_books["book_id"] = my_books["book_id"].astype(str)

In [8]:
my_books

,user_id,book_id,rating,title
0,-1,2517439,5,"The Forever War (The Forever War, #1)"
1,-1,113576,5,The Smartest Guys in the Room: The Amazing Ris...
2,-1,35100,5,Battle Cry of Freedom
3,-1,228221,5,The Mask of Command
5,-1,17662739,5,"2001: A Space Odyssey (Space Odyssey, #1)"
6,-1,356824,5,India After Gandhi: The History of the World's...
7,-1,12125412,5,The Lady or the Tiger?: and Other Logic Puzzles
8,-1,139069,5,Endurance: Shackleton's Incredible Voyage
10,-1,76680,5,"Foundation (Foundation, #1)"
11,-1,1898,5,Into Thin Air: A Personal Account of the Mount...


In [9]:
my_books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 532
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  27 non-null     int64 
 1   book_id  27 non-null     object
 2   rating   27 non-null     int64 
 3   title    27 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.1+ KB


### Finding similarities

In [10]:
!head book_id_map.csv

book_id_csv,book_id
0,34684622
1,34536488
2,34017076
3,71730
4,30422361
5,33503613
6,33517540
7,34467031
8,6383669


In [11]:
# create csv_book_mapping by reading line by line (streaming)
csv_book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

In [12]:
# create set to find book id's
book_set = set(my_books["book_id"])

In [13]:
!ls -lh | grep goodreads_interactions.csv

-rw------- 1 root root 4.1G Mar  4 18:56 goodreads_interactions.csv


In [14]:
!wc -l goodreads_interactions.csv

228648343 goodreads_interactions.csv


In [16]:
!head goodreads_interactions.csv

user_id,book_id,is_read,rating,is_reviewed
0,948,1,5,0
0,947,1,5,1
0,946,1,5,0
0,945,1,5,0
0,944,1,5,0
0,943,1,5,0
0,942,1,5,0
0,941,1,5,0
0,940,1,5,0


In [15]:
# create dict - reading line by line(streaming) to avoid using excess memory

overlap_users = {}

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        book_id = csv_book_mapping.get(csv_id)
        
        if book_id in book_set:
            if user_id not in overlap_users:
                overlap_users[user_id] = 1
            else:
                overlap_users[user_id] += 1

In [17]:
len(overlap_users)

316341

In [18]:
# find users that have read same books

filtered_overlap_users = set([k for k in overlap_users if overlap_users[k] > my_books.shape[0]/5])

In [19]:
filtered_overlap_users

{'342010',
 '149655',
 '238494',
 '243659',
 '21548',
 '440914',
 '111126',
 '188044',
 '413655',
 '257861',
 '133140',
 '380750',
 '428890',
 '375494',
 '321870',
 '274426',
 '289977',
 '1252',
 '8640',
 '165140',
 '439423',
 '178873',
 '118262',
 '395198',
 '92128',
 '47197',
 '254583',
 '8358',
 '422386',
 '158745',
 '65607',
 '152232',
 '348575',
 '333369',
 '72573',
 '362948',
 '142485',
 '113076',
 '18666',
 '352659',
 '410308',
 '325795',
 '208320',
 '71517',
 '81181',
 '36497',
 '158164',
 '277413',
 '391354',
 '378098',
 '34085',
 '56922',
 '151679',
 '171173',
 '259787',
 '12497',
 '277922',
 '428726',
 '156384',
 '420893',
 '409834',
 '435763',
 '192513',
 '186392',
 '196843',
 '389057',
 '68711',
 '22239',
 '213917',
 '219453',
 '392180',
 '23609',
 '256694',
 '176729',
 '287758',
 '317775',
 '48578',
 '32545',
 '353492',
 '169303',
 '410594',
 '184772',
 '341674',
 '89803',
 '401224',
 '49117',
 '372715',
 '21562',
 '278856',
 '219620',
 '146352',
 '143017',
 '66948',
 '28

In [20]:
len(filtered_overlap_users)

1258

### Finding similar user book ratings

In [21]:
interactions_list = []

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in filtered_overlap_users:
            book_id = csv_book_mapping[csv_id]
            interactions_list.append([user_id, book_id, rating])

In [22]:
len(interactions_list)

5638701

In [23]:
interactions_list[0]

['282', '627206', '4']

### Create user-book matrix

In [24]:
interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "rating"])

In [25]:
# add our own ratings from 'my_books'
interactions = pd.concat([my_books[["user_id", "book_id", "rating"]], interactions])

In [26]:
interactions

,user_id,book_id,rating
0,-1,2517439,5
1,-1,113576,5
2,-1,35100,5
3,-1,228221,5
5,-1,17662739,5
...,...,...,...
5638696,804100,475178,0
5638697,804100,186074,0
5638698,804100,153008,0
5638699,804100,45107,0


In [27]:
# set dtypes

interactions["book_id"] = interactions["book_id"].astype(str)
interactions["user_id"] = interactions["user_id"].astype(str)
interactions["rating"] = pd.to_numeric(interactions["rating"])

In [28]:
interactions["user_id"].unique()

array(['-1', '282', '874', ..., '442043', '712588', '804100'],
      dtype=object)

In [29]:
interactions["user_index"] = interactions["user_id"].astype("category").cat.codes

In [30]:
interactions.iloc[0]

user_id            -1
book_id       2517439
rating              5
user_index          0
Name: 0, dtype: object

In [31]:
interactions.iloc[2000]

user_id        874
book_id       5308
rating           3
user_index    1216
Name: 1973, dtype: object

In [32]:
interactions["user_index"].unique()

array([   0,  555, 1216, ..., 1054, 1143, 1183], dtype=int16)

In [37]:
len(interactions["user_index"].unique())

1259

In [33]:
interactions["book_index"] = interactions["book_id"].astype("category").cat.codes

In [34]:
interactions.iloc[0]

user_id            -1
book_id       2517439
rating              5
user_index          0
book_index     414880
Name: 0, dtype: object

In [35]:
interactions["book_index"].unique()

array([414880,  38971, 575858, ..., 759827, 631564, 552277], dtype=int32)

In [36]:
len(interactions["book_index"].unique())

802870

In [38]:
1259 * 802870

1010813330

In [39]:
interactions

,user_id,book_id,rating,user_index,book_index
0,-1,2517439,5,0,414880
1,-1,113576,5,0,38971
2,-1,35100,5,0,575858
3,-1,228221,5,0,356004
5,-1,17662739,5,0,214285
...,...,...,...,...,...
5638696,804100,475178,0,1183,617107
5638697,804100,186074,0,1183,258768
5638698,804100,153008,0,1183,141428
5638699,804100,45107,0,1183,611284


In [40]:
# use id to position - sparse vs dense matrix
# use sparse to use less memory

from scipy.sparse import coo_matrix

ratings_mat_coo = coo_matrix((interactions["rating"], (interactions["user_index"], interactions["book_index"])))

In [42]:
ratings_mat_coo

<1259x802870 sparse matrix of type '<class 'numpy.int64'>'
	with 5638728 stored elements in COOrdinate format>

In [41]:
ratings_mat_coo.shape

(1259, 802870)

In [43]:
ratings_mat = ratings_mat_coo.tocsr()

### Finding users similar to us

In [44]:
# convert from coo to csr format
interactions[interactions["user_id"] == "-1"]

,user_id,book_id,rating,user_index,book_index
0,-1,2517439,5,0,414880
1,-1,113576,5,0,38971
2,-1,35100,5,0,575858
3,-1,228221,5,0,356004
5,-1,17662739,5,0,214285
6,-1,356824,5,0,581743
7,-1,12125412,5,0,59763
8,-1,139069,5,0,124430
10,-1,76680,5,0,722098
11,-1,1898,5,0,276178


In [45]:
my_index = 0

In [47]:
# use cosign similarity

from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[my_index,:], ratings_mat).flatten()

In [48]:
similarity[0]

0.9999999999999999

In [49]:
similarity[1]

0.04579825781910479

In [51]:
# find 15 users who are most similar to us
import numpy as np

indices = np.argpartition(similarity, -15)[-15:]

In [52]:
indices

array([1188,  942,  218,  129,  496,  435, 1208,  795, 1213, 1210, 1143,
        321,  294,  862,    0])

In [53]:
# find their userids
similar_users = interactions[interactions["user_index"].isin(indices)].copy()

In [57]:
# take ourselves out to check other similar users only
similar_users = similar_users[similar_users["user_id"] != "-1"]

In [58]:
similar_users

,user_id,book_id,rating,user_index,book_index
45312,4133,5359,3,942,632143
45313,4133,10464963,4,942,13492
45314,4133,3858,3,942,593622
45315,4133,11827808,4,942,51904
45316,4133,7913305,4,942,732465
...,...,...,...,...,...
5638521,712588,32388712,3,1143,543119
5638522,712588,16322,5,1143,183365
5638523,712588,860543,0,1143,759827
5638524,712588,853510,5,1143,756768


### Creating book recommendations

In [59]:
# find how many times book appeared
book_recs = similar_users.groupby("book_id").rating.agg(['count', 'mean'])

In [60]:
book_recs

,count,mean
book_id,,
1,6,3.833333
100322,1,0.000000
100365,1,0.000000
10046142,1,0.000000
1005,3,0.000000
...,...,...
99561,2,2.500000
99610,1,3.000000
99664,1,4.000000


In [61]:
# read in books titles json file
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [62]:
# merge 2 datasets to get books titles into our recommendations
book_recs = book_recs.merge(books_titles, how="inner", on="book_id")

In [63]:
book_recs

,book_id,count,mean,title,ratings,url,image_url,mod_title
0,1,6,3.833333,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...,https://images.gr-assets.com/books/1361039191m...,harry potter and the halfblood prince harry po...
1,100322,1,0.000000,Assata: An Autobiography,11057,https://www.goodreads.com/book/show/100322.Assata,https://images.gr-assets.com/books/1328857268m...,assata an autobiography
2,100365,1,0.000000,The Mote in God's Eye,48736,https://www.goodreads.com/book/show/100365.The...,https://images.gr-assets.com/books/1399490037m...,the mote in gods eye
3,10046142,1,0.000000,Dancing in the Glory of Monsters: The Collapse...,2391,https://www.goodreads.com/book/show/10046142-d...,https://images.gr-assets.com/books/1328757755m...,dancing in the glory of monsters the collapse ...
4,1005,3,0.000000,Think and Grow Rich,87634,https://www.goodreads.com/book/show/1005.Think...,https://s.gr-assets.com/assets/nophoto/book/11...,think and grow rich
...,...,...,...,...,...,...,...,...
2843,99561,2,2.500000,Looking for Alaska,804587,https://www.goodreads.com/book/show/99561.Look...,https://images.gr-assets.com/books/1394798630m...,looking for alaska
2844,99610,1,3.000000,The Best Laid Plans,17434,https://www.goodreads.com/book/show/99610.The_...,https://images.gr-assets.com/books/1353374848m...,the best laid plans
2845,99664,1,4.000000,The Painted Veil,24606,https://www.goodreads.com/book/show/99664.The_...,https://images.gr-assets.com/books/1320421719m...,the painted veil
2846,9969571,3,2.333333,Ready Player One,376328,https://www.goodreads.com/book/show/9969571-re...,https://images.gr-assets.com/books/1500930947m...,ready player one


### Ranking our book recommendations

In [66]:
# create adjusted_count - includes count and ratings
book_recs["adjusted_count"] = book_recs["count"] * (book_recs["count"] / book_recs["ratings"])

In [67]:
# create mean score
book_recs["score"] = book_recs["mean"] * book_recs["adjusted_count"]

In [68]:
# take out books that we already read
book_recs = book_recs[~book_recs["book_id"].isin(my_books["book_id"])]

In [69]:
book_recs

,book_id,count,mean,title,ratings,url,image_url,mod_title,adjusted_count,score
0,1,6,3.833333,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...,https://images.gr-assets.com/books/1361039191m...,harry potter and the halfblood prince harry po...,0.000021,0.000081
1,100322,1,0.000000,Assata: An Autobiography,11057,https://www.goodreads.com/book/show/100322.Assata,https://images.gr-assets.com/books/1328857268m...,assata an autobiography,0.000090,0.000000
2,100365,1,0.000000,The Mote in God's Eye,48736,https://www.goodreads.com/book/show/100365.The...,https://images.gr-assets.com/books/1399490037m...,the mote in gods eye,0.000021,0.000000
3,10046142,1,0.000000,Dancing in the Glory of Monsters: The Collapse...,2391,https://www.goodreads.com/book/show/10046142-d...,https://images.gr-assets.com/books/1328757755m...,dancing in the glory of monsters the collapse ...,0.000418,0.000000
4,1005,3,0.000000,Think and Grow Rich,87634,https://www.goodreads.com/book/show/1005.Think...,https://s.gr-assets.com/assets/nophoto/book/11...,think and grow rich,0.000103,0.000000
...,...,...,...,...,...,...,...,...,...,...
2843,99561,2,2.500000,Looking for Alaska,804587,https://www.goodreads.com/book/show/99561.Look...,https://images.gr-assets.com/books/1394798630m...,looking for alaska,0.000005,0.000012
2844,99610,1,3.000000,The Best Laid Plans,17434,https://www.goodreads.com/book/show/99610.The_...,https://images.gr-assets.com/books/1353374848m...,the best laid plans,0.000057,0.000172
2845,99664,1,4.000000,The Painted Veil,24606,https://www.goodreads.com/book/show/99664.The_...,https://images.gr-assets.com/books/1320421719m...,the painted veil,0.000041,0.000163
2846,9969571,3,2.333333,Ready Player One,376328,https://www.goodreads.com/book/show/9969571-re...,https://images.gr-assets.com/books/1500930947m...,ready player one,0.000024,0.000056


In [70]:
# create mod_title 
my_books["mod_title"] = my_books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()

In [71]:
# replace sequences of spaces
my_books["mod_title"] = my_books["mod_title"].str.replace("\s+", " ", regex=True)

In [72]:
# take out anything into rec where mod_title fits
book_recs = book_recs[~book_recs["mod_title"].isin(my_books["mod_title"])]

In [73]:
# remove anything that has count mean >=4 and count >2
book_recs = book_recs[book_recs["mean"] >=4]
book_recs = book_recs[book_recs["count"]>2]

In [74]:
# create top recommendations
top_recs = book_recs.sort_values("mean", ascending=False)

In [75]:
top_recs

,book_id,count,mean,title,ratings,url,image_url,mod_title,adjusted_count,score
2260,62291,5,4.800000,"A Storm of Swords (A Song of Ice and Fire, #3)",477834,https://www.goodreads.com/book/show/62291.A_St...,https://images.gr-assets.com/books/1497931121m...,a storm of swords a song of ice and fire 3,0.000052,0.000251
600,157993,3,4.333333,The Little Prince,763309,https://www.goodreads.com/book/show/157993.The...,https://images.gr-assets.com/books/1367545443m...,the little prince,0.000012,0.000051
1100,22034,3,4.333333,The Godfather,259150,https://www.goodreads.com/book/show/22034.The_...,https://images.gr-assets.com/books/1394988109m...,the godfather,0.000035,0.000150
1173,2318271,3,4.333333,The Last Lecture,245804,https://www.goodreads.com/book/show/2318271.Th...,https://images.gr-assets.com/books/1388075896m...,the last lecture,0.000037,0.000159
1906,4381,3,4.333333,Fahrenheit 451,591506,https://www.goodreads.com/book/show/4381.Fahre...,https://images.gr-assets.com/books/1351643740m...,fahrenheit 451,0.000015,0.000066
243,119322,4,4.250000,"The Golden Compass (His Dark Materials, #1)",973154,https://www.goodreads.com/book/show/119322.The...,https://images.gr-assets.com/books/1505766203m...,the golden compass his dark materials 1,0.000016,0.000070
1441,2767793,4,4.250000,"The Hero of Ages (Mistborn, #3)",149260,https://www.goodreads.com/book/show/2767793-th...,https://images.gr-assets.com/books/1480717763m...,the hero of ages mistborn 3,0.000107,0.000456
2558,78983,4,4.250000,"Kane and Abel (Kane and Abel, #1)",75215,https://www.goodreads.com/book/show/78983.Kane...,https://s.gr-assets.com/assets/nophoto/book/11...,kane and abel kane and abel 1,0.000213,0.000904
244,119324,3,4.000000,"The Subtle Knife (His Dark Materials, #2)",246697,https://www.goodreads.com/book/show/119324.The...,https://images.gr-assets.com/books/1505766360m...,the subtle knife his dark materials 2,0.000036,0.000146
398,13497,4,4.000000,"A Feast for Crows (A Song of Ice and Fire, #4)",437398,https://www.goodreads.com/book/show/13497.A_Fe...,https://images.gr-assets.com/books/1429538615m...,a feast for crows a song of ice and fire 4,0.000037,0.000146


In [76]:
# create functions to get clickable links and render cover images
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

top_recs.style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,count,mean,title,ratings,url,image_url,mod_title,adjusted_count,score
2260,62291,5,4.800000,"A Storm of Swords (A Song of Ice and Fire, #3)",477834,Goodreads,https://images.gr-assets.com/books/1497931121m/62291.jpg,a storm of swords a song of ice and fire 3,0.000052,0.000251
600,157993,3,4.333333,The Little Prince,763309,Goodreads,https://images.gr-assets.com/books/1367545443m/157993.jpg,the little prince,0.000012,0.000051
1100,22034,3,4.333333,The Godfather,259150,Goodreads,https://images.gr-assets.com/books/1394988109m/22034.jpg,the godfather,0.000035,0.000150
1173,2318271,3,4.333333,The Last Lecture,245804,Goodreads,https://images.gr-assets.com/books/1388075896m/2318271.jpg,the last lecture,0.000037,0.000159
1906,4381,3,4.333333,Fahrenheit 451,591506,Goodreads,https://images.gr-assets.com/books/1351643740m/4381.jpg,fahrenheit 451,0.000015,0.000066
243,119322,4,4.250000,"The Golden Compass (His Dark Materials, #1)",973154,Goodreads,https://images.gr-assets.com/books/1505766203m/119322.jpg,the golden compass his dark materials 1,0.000016,0.000070
1441,2767793,4,4.250000,"The Hero of Ages (Mistborn, #3)",149260,Goodreads,https://images.gr-assets.com/books/1480717763m/2767793.jpg,the hero of ages mistborn 3,0.000107,0.000456
2558,78983,4,4.250000,"Kane and Abel (Kane and Abel, #1)",75215,Goodreads,https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png,kane and abel kane and abel 1,0.000213,0.000904
244,119324,3,4.000000,"The Subtle Knife (His Dark Materials, #2)",246697,Goodreads,https://images.gr-assets.com/books/1505766360m/119324.jpg,the subtle knife his dark materials 2,0.000036,0.000146
398,13497,4,4.000000,"A Feast for Crows (A Song of Ice and Fire, #4)",437398,Goodreads,https://images.gr-assets.com/books/1429538615m/13497.jpg,a feast for crows a song of ice and fire 4,0.000037,0.000146


### Next steps

1. filter users less or more in 'filtered_overlap_users' --> to get more books data to base recommendations on
2. Scrape and get more books into your 'liked_books.csv'
3. Try find more similar users e. g. 100 users instead of 15
4. Add Weights to recommendations
5. Tewak details in how we create score